## 기본 세팅

### LangSmith 설정
- 반응 추적

#### Ref
- [테디노트](https://wikidocs.net/250954)

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "PDF Chatbot with Chroma & groq "
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")